# 🚀 Alertas de Gols no WhatsApp

In [ ]:

!pip install requests

import requests
import time
from datetime import datetime

# Suas credenciais
API_FOOTBALL_KEY = "6244442c30590171826e72671858f6e2"
WHATSAPP_TOKEN = "EAAoXQSWs8lkBPKgrLQm9ZBs8C7H4CUeJdsdRDrxOupVrss1d4OIJy039eldKh7U3dVjezuvflm68INm2UkK3MS2LPZBRy0q7rEo8ebP0kBrO6x1NpDdNzZB684l7AcD4O3w2LtEKuWMqHwqcnO0WpAPGO7oxnqZACakSEBSUrKIxNyCBxXZBwSZCVuSW2KpMg1hjBNZADHH4diuf1LdwZBMZCdZBaK31NvasKu01xj54JEnDxudQZDZD"
WHATSAPP_PHONE_ID = "721571921044564"
DESTINATION_PHONE = "5531998289971"

LEAGUE_IDS = {
    "Inglaterra": 39,
    "Noruega": 103,
    "Alemanha": 78,
    "Itália": 135,
    "Portugal": 94,
    "França": 61,
    "Brasil": 71,
    "Argentina": 128,
    "Espanha": 140,
    "Libertadores": 131,
    "Sul-Americana": 134
}

def send_whatsapp_message(message):
    url = f"https://graph.facebook.com/v19.0/{WHATSAPP_PHONE_ID}/messages"
    headers = {
        "Authorization": f"Bearer {WHATSAPP_TOKEN}",
        "Content-Type": "application/json"
    }
    payload = {
        "messaging_product": "whatsapp",
        "to": DESTINATION_PHONE,
        "type": "text",
        "text": {"body": message}
    }
    response = requests.post(url, json=payload, headers=headers)
    print(f"[WhatsApp] Enviado: {message} | Status: {response.status_code}")

def get_match_stats(fixture_id):
    headers = {"x-apisports-key": API_FOOTBALL_KEY}
    params = {"fixture": fixture_id}
    try:
        r = requests.get("https://v3.football.api-sports.io/fixtures/statistics", headers=headers, params=params)
        data = r.json().get("response", [])
        stats = {}
        for team_stats in data:
            team_name = team_stats["team"]["name"]
            stats[team_name] = {}
            for item in team_stats["statistics"]:
                stats[team_name][item["type"]] = item["value"]
        return stats
    except Exception as e:
        print(f"[Erro Stats] {e}")
        return {}

def check_live_games():
    headers = {"x-apisports-key": API_FOOTBALL_KEY}
    for league, league_id in LEAGUE_IDS.items():
        params = {"live": "all", "league": league_id, "season": 2024}
        try:
            r = requests.get("https://v3.football.api-sports.io/fixtures", headers=headers, params=params)
            matches = r.json().get("response", [])
            for match in matches:
                fixture_id = match["fixture"]["id"]
                home = match["teams"]["home"]["name"]
                away = match["teams"]["away"]["name"]
                goals_home = match["goals"]["home"]
                goals_away = match["goals"]["away"]

                stats = get_match_stats(fixture_id)

                if stats:
                    home_shots = stats[home].get("Total Shots", 0)
                    away_shots = stats[away].get("Total Shots", 0)
                    home_attacks = stats[home].get("Dangerous Attacks", 0)
                    away_attacks = stats[away].get("Dangerous Attacks", 0)

                    if (home_shots + away_shots >= 8 or home_attacks + away_attacks >= 20) and \
                       (goals_home + goals_away) < 3:
                        minute = match["fixture"]["status"]["elapsed"]
                        message = (
                            f"⚽ ALERTA DE GOL ⚽\n"
                            f"{home} x {away} ({goals_home}-{goals_away}) - {league}\n"
                            f"Minuto: {minute}'\n"
                            f"Chutes: {home_shots + away_shots}\n"
                            f"Ataques perigosos: {home_attacks + away_attacks}"
                        )
                        send_whatsapp_message(message)
        except Exception as e:
            print(f"[Erro Jogos] {e}")

if __name__ == "__main__":
    print("🚀 Sistema de alertas iniciado...")
    while True:
        check_live_games()
        time.sleep(60)  # a cada 1 minuto


🚀 Sistema de alertas iniciado...
